<h1> pre-processing and preparing the train data </h1>



<p> 
    <i>
     <ul>
  <li>In this phase we are going to do an image processing task</li>
  <li>first,we have to split each sudoku puzzle image in our train directory to 81 cells (81 sub images) and append each      digit's image to a list.</li>
  <li>At the end of this phase we get a list of digit's images that should be used for training, for example if  the number of         the Training images is 150 we get a list with 150 * 81 = 12150 digit's image</li>
</ul>
    </i>
</p>

In [1]:
# functions rectify and build_train_data are responsible for splitting the sudoku puzzle to  to cells (sub images) each cell contains one digit of the puzzle

import cv2
import numpy as np 
import pytesseract
from skimage.segmentation import clear_border
import os



def rectify(h):
        h = h.reshape((4,2))
        hnew = np.zeros((4,2),dtype = np.float32)

        add = h.sum(1)
        hnew[0] = h[np.argmin(add)]
        hnew[2] = h[np.argmax(add)]

        diff = np.diff(h,axis = 1)
        hnew[1] = h[np.argmin(diff)]
        hnew[3] = h[np.argmax(diff)]

        return hnew

# detect  each puzzle and split it into 81 sub images (corresponds to each digit in the cell) 

def build_train_data(path : str):    
    
    dirs = os.listdir( path ) # the path of  directory which contains all the training images
    liste4 = []
       
    for file in dirs:
    
            img=cv2.imread(os.path.join(path,file))
            if (img.shape[0] > 400 ):
                img = img = cv2.resize(img,(300,300))
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

            gray = cv2.GaussianBlur(gray,(5,5),0)
            thresh = cv2.adaptiveThreshold(gray,255,1,1,11,2)


            contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            biggest = None
            max_area = 0
            for i in contours:
                area = cv2.contourArea(i)
                if area > 100:
                        peri = cv2.arcLength(i,True)
                        approx = cv2.approxPolyDP(i,0.02*peri,True)
                        if area > max_area and len(approx)==4:
                                biggest = approx
                                max_area = area
            cv2.drawContours(img, biggest, -1, (0,255,0), 8)



            biggest=rectify(biggest)

            h = np.array([ [0,0],[449,0],[449,449],[0,449] ],np.float32)
            retval = cv2.getPerspectiveTransform(biggest,h)
            thresh = cv2.adaptiveThreshold(gray,255,1,1,11,2)

            warp = cv2.warpPerspective(thresh,retval,(450,450))  # contains the sudoku puzzle 

            winX = int(warp.shape[1]/9.0)
            winY = int(warp.shape[0]/9.0)
 
            
            #loop through all the cells of the sudoku puzzle and append

            for y in range(0,warp.shape[0],winY):
                  for x in range(0,warp.shape[1],winX):
                #slice the cell
                        window = warp[y:y+winY,x:x+winX]
                #sanity check
                        if window.shape[0] != winY or window.shape[1] != winX:
                          continue
                        clone = warp.copy()
                        digit = cv2.resize(window,(28,28))
                #clear borders
                        digit = clear_border(digit)
                        liste4.append(digit)
                       
            
    return liste4

<p> 
    <i>
     <ul>
  <li>after getting a list of all the digit's images , now we have to get all the labels for those images ( label in this case is  the value of the digit in each cell )</li>
  <li> to get the label of each digit's image in the training data , we have to read each image metadata file (contains the corresponding matrix of each sudoku puzzle image) and append the content of that matrix to a list.</li>
  <li>At the end of this phase we get a list of digits, each element of the list is the label(value) of the digit in the corresponding cell of our sudoku puzzles</li>
</ul>
    </i>
</p>

In [2]:
# read  labels of our training images 
def build_get_labels(path : str):
    dirs = os.listdir( path )  # path of directory which contains all the metadata files  
    liste1 = []
    # This would print all the files and directories
    for file in dirs:
        temp = open(os.path.join(path,file),'r').read().split('\n')

        del temp[0]
        del temp[0]
        del temp[len(temp)-1]
        for i in temp : 
            liste = i.split(" ")
            for j in liste :
                if j != "":
                    liste1.append(int(j))
    return liste1

In [3]:
#y_train contains all the labels of our training images 

y_train = build_get_labels("train/label_image") 

#x_train contains all the training images 

x_train = build_train_data("train/train_image") 

In [4]:
#reshaping the training data to feed our model 

x_train = np.asarray(x_train)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value
x_train /= 255

y_train = np.asarray(y_train)

<h1>build and train the model</h1>

<p> 
   <i> to build our model i used Keras API which is a high-level neural networks API,running on top of TensorFlow,
       and is also considered to be the most straightforward TensorFlow API.</i>
</p>

In [5]:
# Importing the required Keras modules 
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


In [6]:
# Creating a Sequential Model and adding the layers
model = Sequential()

model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = (28, 28, 1)))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.2)) 
model.add(Dense(10, activation = 'softmax'))

In [7]:
#Compiling and Fitting the Model

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=25)

Epoch 1/25
12879/12879 [==============================] - 12s 896us/step - loss: 0.5518 - accuracy: 0.8256
Epoch 2/25
12879/12879 [==============================] - 11s 856us/step - loss: 0.1753 - accuracy: 0.9581
Epoch 3/25
12879/12879 [==============================] - 12s 899us/step - loss: 0.1219 - accuracy: 0.9709
Epoch 4/25
12879/12879 [==============================] - 14s 1ms/step - loss: 0.1036 - accuracy: 0.9765
Epoch 5/25
12879/12879 [==============================] - 12s 956us/step - loss: 0.0936 - accuracy: 0.9769
Epoch 6/25
12879/12879 [==============================] - 11s 887us/step - loss: 0.0838 - accuracy: 0.9791
Epoch 7/25
12879/12879 [==============================] - 12s 926us/step - loss: 0.0811 - accuracy: 0.9788
Epoch 8/25
12879/12879 [==============================] - 13s 994us/step - loss: 0.0726 - accuracy: 0.9818
Epoch 9/25
12879/12879 [==============================] - 13s 1ms/step - loss: 0.0652 - accuracy: 0.9832
Epoch 10/25
12879/12879 [================

<h1> Save the created Model </h1>

In [73]:
# save the Model 

model.save('model_sudoku.h5')